# FINMA Python Lab 07: File Input/Output

## Overview

In this lab, you'll practice:
- Reading text files (.txt)
- Reading markdown files (.md)
- Reading CSV files (.csv)
- Writing to text files
- Writing to CSV files
- Processing financial data from files
- File handling best practices

**Important:** Complete your work and have it manually checked by your instructor.

---

## Part 1: Reading Text Files

### Basic File Reading

The basic syntax for reading a file:

```python
# Method 1: Manual close
file = open('filename.txt', 'r')
content = file.read()
file.close()

# Method 2: Using 'with' (recommended - auto-closes)
with open('filename.txt', 'r') as file:
    content = file.read()
```

### File Modes:
- `'r'` - Read (default)
- `'w'` - Write (overwrites)
- `'a'` - Append
- `'r+'` - Read and write

### Example: Reading a Market Report

In [ ]:
# Read entire file as string
with open('sample_data/market_report.txt', 'r') as file:
    content = file.read()
    print(content)

### Reading Line by Line

In [ ]:
# Read file line by line
with open('sample_data/market_report.txt', 'r') as file:
    for line_number, line in enumerate(file, 1):
        print(f"Line {line_number}: {line.strip()}")

### Reading All Lines into a List

In [ ]:
# Read all lines into a list
with open('sample_data/market_report.txt', 'r') as file:
    lines = file.readlines()

print(f"Total lines: {len(lines)}")
print(f"\nFirst 5 lines:")
for line in lines[:5]:
    print(line.strip())

---

## Part 2: Reading Markdown Files

Markdown files are text files with special formatting. We can read them like any text file.

In [ ]:
# Read markdown file
with open('sample_data/portfolio_report.md', 'r') as file:
    content = file.read()
    print(content)

### Extracting Specific Information

In [ ]:
# Extract headers from markdown
with open('sample_data/portfolio_report.md', 'r') as file:
    headers = []
    for line in file:
        if line.startswith('#'):
            headers.append(line.strip())

print("Headers found:")
for header in headers:
    print(header)

---

## Part 3: Reading CSV Files

CSV (Comma-Separated Values) files are commonly used for tabular data.

### Method 1: Using the csv Module

In [ ]:
import csv

# Read CSV file
with open('sample_data/portfolio.csv', 'r') as file:
    csv_reader = csv.reader(file)
    
    # Get header
    header = next(csv_reader)
    print("Columns:", header)
    print()
    
    # Read data rows
    print("Portfolio Holdings:")
    for row in csv_reader:
        print(f"{row[0]:6} | {row[1]:25} | ${float(row[3]):8.2f} | {row[4]:3} shares")

### Method 2: Using csv.DictReader

In [ ]:
import csv

# Read CSV as dictionaries
with open('sample_data/portfolio.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    
    print("Portfolio Summary:")
    print("=" * 70)
    
    total_value = 0
    for row in csv_reader:
        symbol = row['symbol']
        company = row['company']
        value = float(row['value'])
        total_value += value
        
        print(f"{symbol:6} | {company:25} | ${value:12,.2f}")
    
    print("=" * 70)
    print(f"{'TOTAL':6} | {'':25} | ${total_value:12,.2f}")

### Reading CSV into a List of Dictionaries

In [ ]:
import csv

def read_portfolio(filename):
    """Read portfolio from CSV file"""
    portfolio = []
    
    with open(filename, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            # Convert numeric fields
            row['price'] = float(row['price'])
            row['shares'] = int(row['shares'])
            row['value'] = float(row['value'])
            portfolio.append(row)
    
    return portfolio

# Load portfolio
portfolio = read_portfolio('sample_data/portfolio.csv')

# Show first 3 entries
print("First 3 holdings:")
for holding in portfolio[:3]:
    print(holding)

---

## Part 4: Writing to Text Files

### Basic File Writing

In [ ]:
# Write to a new file
with open('output/my_report.txt', 'w') as file:
    file.write("FINMA Portfolio Report\n")
    file.write("=" * 40 + "\n")
    file.write("Date: January 4, 2026\n\n")
    file.write("Total Portfolio Value: $111,089.65\n")

print("File written successfully!")

# Read it back to verify
with open('output/my_report.txt', 'r') as file:
    print(file.read())

### Appending to Files

In [ ]:
# Append to existing file
with open('output/my_report.txt', 'a') as file:
    file.write("\nTop Holdings:\n")
    file.write("- MSFT: $28,418.25\n")
    file.write("- AAPL: $15,075.00\n")
    file.write("- NVDA: $12,380.50\n")

# Read the updated file
with open('output/my_report.txt', 'r') as file:
    print(file.read())

---

## Part 5: Writing to CSV Files

### Method 1: Using csv.writer

In [ ]:
import csv

# Sample data
trades = [
    ['Date', 'Symbol', 'Action', 'Shares', 'Price', 'Total'],
    ['2026-01-02', 'AAPL', 'BUY', 100, 150.75, 15075.00],
    ['2026-01-02', 'GOOGL', 'BUY', 50, 138.21, 6910.50],
    ['2026-01-03', 'MSFT', 'BUY', 75, 378.91, 28418.25]
]

# Write to CSV
with open('output/trades.csv', 'w', newline='') as file:
    csv_writer = csv.writer(file)
    csv_writer.writerows(trades)

print("CSV file written!")

# Read it back
with open('output/trades.csv', 'r') as file:
    print(file.read())

### Method 2: Using csv.DictWriter

In [ ]:
import csv

# Sample data as dictionaries
holdings = [
    {'symbol': 'AAPL', 'shares': 100, 'price': 150.75},
    {'symbol': 'GOOGL', 'shares': 50, 'price': 138.21},
    {'symbol': 'MSFT', 'shares': 75, 'price': 378.91}
]

# Write to CSV
with open('output/holdings.csv', 'w', newline='') as file:
    fieldnames = ['symbol', 'shares', 'price']
    csv_writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    csv_writer.writeheader()
    csv_writer.writerows(holdings)

print("CSV file written with DictWriter!")

# Read it back
with open('output/holdings.csv', 'r') as file:
    print(file.read())

---

## Part 6: Processing and Transforming Data

### Example: Calculate Portfolio Statistics

In [ ]:
import csv

def analyze_portfolio(input_file, output_file):
    """Analyze portfolio and write summary to file"""
    
    # Read portfolio
    holdings = []
    with open(input_file, 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            row['value'] = float(row['value'])
            holdings.append(row)
    
    # Calculate statistics
    total_value = sum(h['value'] for h in holdings)
    num_holdings = len(holdings)
    avg_position = total_value / num_holdings
    
    # Group by sector
    sectors = {}
    for holding in holdings:
        sector = holding['sector']
        if sector not in sectors:
            sectors[sector] = 0
        sectors[sector] += holding['value']
    
    # Write summary
    with open(output_file, 'w') as file:
        file.write("PORTFOLIO ANALYSIS REPORT\n")
        file.write("=" * 50 + "\n\n")
        
        file.write(f"Total Value: ${total_value:,.2f}\n")
        file.write(f"Number of Holdings: {num_holdings}\n")
        file.write(f"Average Position Size: ${avg_position:,.2f}\n\n")
        
        file.write("Sector Allocation:\n")
        file.write("-" * 50 + "\n")
        for sector, value in sorted(sectors.items(), key=lambda x: x[1], reverse=True):
            percentage = (value / total_value) * 100
            file.write(f"{sector:25} ${value:12,.2f} ({percentage:5.2f}%)\n")
    
    print(f"Analysis written to {output_file}")

# Run analysis
analyze_portfolio('sample_data/portfolio.csv', 'output/portfolio_analysis.txt')

# Display the result
with open('output/portfolio_analysis.txt', 'r') as file:
    print(file.read())

---

## Programming Exercises

### Exercise 1: Count Words in Market Report

Write a function that:
1. Reads the market report text file
2. Counts the total number of words
3. Counts how many times specific keywords appear ("market", "sector", "trading")
4. Prints the results

**Expected output format:**
```
Total words: 123
Keyword counts:
  market: 5
  sector: 3
  trading: 2
```

In [ ]:
# Exercise 1: Count Words in Market Report
# Write your code here

def word_counter()


### Exercise 2: Extract Holdings from Markdown

Write a function that:
1. Reads the portfolio report markdown file
2. Extracts all stock symbols (hint: they appear with ** bold markers)
3. Returns a list of symbols
4. Prints the symbols

**Expected output:**
```
Holdings found: ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
```

In [ ]:
# Exercise 2: Extract Holdings from Markdown
# Write your code here



### Exercise 3: Calculate Sector Totals

Write a function that:
1. Reads the portfolio CSV file
2. Groups holdings by sector
3. Calculates total value for each sector
4. Returns a dictionary: `{sector: total_value}`
5. Prints the results sorted by value (descending)

**Expected output format:**
```
Technology: $55,873.75
Financials: $19,153.95
Consumer Discretionary: $14,419.20
...
```

In [ ]:
# Exercise 3: Calculate Sector Totals
# Write your code here



### Exercise 4: Price Change Analysis

Write a function that:
1. Reads the stock_prices.csv file
2. For each stock symbol, calculates the price change from first to last day
3. Calculates the percentage change
4. Writes results to a new CSV file with columns: symbol, start_price, end_price, change, change_pct

**Hint:** You'll need to track the first and last price for each symbol.

In [ ]:
# Exercise 4: Price Change Analysis
# Write your code here



### Exercise 5: Portfolio Performance Report

Write a function that:
1. Reads the portfolio.csv file
2. Calculates:
   - Total portfolio value
   - Largest position (by value)
   - Smallest position (by value)
   - Average position size
   - Number of positions
3. Writes a formatted report to a text file

**Output file format:**
```
PORTFOLIO PERFORMANCE REPORT
============================
Generated: [current date]

Summary Statistics:
  Total Value: $XXX,XXX.XX
  Number of Positions: XX
  Average Position: $XX,XXX.XX

Extremes:
  Largest: SYMBOL ($XX,XXX.XX)
  Smallest: SYMBOL ($X,XXX.XX)
```

In [ ]:
# Exercise 5: Portfolio Performance Report
# Write your code here



### Exercise 6: Filter and Export

Write a function that:
1. Reads the portfolio.csv file
2. Filters holdings based on criteria:
   - Only Technology sector
   - Value greater than $10,000
3. Writes the filtered results to a new CSV file
4. Returns the count of filtered holdings

**Function signature:**
```python
def filter_portfolio(input_file, output_file, sector, min_value):
    # Your code here
    return count
```

In [ ]:
# Exercise 6: Filter and Export
# Write your code here



### Exercise 7: Combine Multiple Files

Write a function that:
1. Reads both portfolio.csv and stock_prices.csv
2. For each holding in the portfolio:
   - Find the latest closing price from stock_prices.csv
   - Calculate the updated position value
   - Calculate the change from original value
3. Writes a new CSV with columns: symbol, shares, original_price, current_price, original_value, current_value, change

**Note:** Use the last entry for each symbol in stock_prices.csv as the current price.

In [ ]:
# Exercise 7: Combine Multiple Files
# Write your code here



### Exercise 8: Trading Log System

Create a complete trading log system with multiple functions:

1. `add_trade(date, symbol, action, shares, price)`: Appends a trade to trades.csv
2. `read_trades(filename)`: Reads all trades from file
3. `calculate_pl(trades)`: Calculates P&L for each symbol
4. `generate_summary(trades, output_file)`: Writes a summary report

**Trades file format:**
```
date,symbol,action,shares,price
2026-01-02,AAPL,BUY,100,150.00
2026-01-03,AAPL,SELL,50,152.00
```

**Summary should include:**
- Total trades
- Total buy volume
- Total sell volume
- P&L by symbol
- Overall P&L

In [ ]:
# Exercise 8: Trading Log System
# Write your code here



---

## Summary: Key Concepts

### Reading Files:
```python
# Text files
with open('file.txt', 'r') as f:
    content = f.read()          # Read entire file
    lines = f.readlines()       # Read as list of lines
    for line in f:              # Iterate line by line
        process(line)

# CSV files
import csv
with open('file.csv', 'r') as f:
    reader = csv.reader(f)      # Read as lists
    reader = csv.DictReader(f)  # Read as dictionaries
```

### Writing Files:
```python
# Text files
with open('file.txt', 'w') as f:
    f.write("text")             # Write string
    f.writelines(lines)         # Write list of strings

# CSV files
import csv
with open('file.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(row)        # Write one row
    writer.writerows(rows)      # Write multiple rows
```

### Best Practices:
1. **Always use `with` statement** - Automatically closes files
2. **Specify encoding** when needed: `open('file.txt', 'r', encoding='utf-8')`
3. **Use `newline=''`** when writing CSV files
4. **Handle exceptions** for file operations
5. **Close files** if not using `with`
6. **Use pathlib** for complex path operations

### File Modes:
- `'r'` - Read (default, file must exist)
- `'w'` - Write (creates new, overwrites existing)
- `'a'` - Append (creates new or adds to existing)
- `'x'` - Exclusive create (fails if file exists)
- `'r+'` - Read and write

---

## Testing and Submission

**Before moving on:**
1. Test all file operations with the sample files
2. Verify output files are created correctly
3. Check CSV files open correctly in Excel/spreadsheet apps
4. Handle edge cases (empty files, missing data)
5. Have your instructor manually check your work

**Great work completing Lab 7!** 🎉